In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os, os.path
import pandas as pd
import temp as tmp
import setup_analysis as sa
import sqlalchemy
import sql_utilities as sqlutil
import support_classes as sc
import support_functions as sf

from typing import *

/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/model_attributes.py:2635: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/model_attributes.py:2392: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for desig, df in df_by_designation:


In [47]:
fp_uncertainty_definitions = "/Users/jsyme/Documents/Projects/git_jbus/sisepuede/ref/ingestion/uncertainty_definitions/uncertainty_definitions_global.xlsx"
df_base = pd.read_excel(fp_uncertainty_definitions)

###################
#    FUNCTIONS    #
###################

def add_value_ranges(
    df: pd.DataFrame,
    dict_scalars_to_val_ranges: Union[Dict, None] = None,
) -> pd.DataFrame:
    
    dict_scalars_to_val_ranges = (
        {
            "min_35": "value_default_min_35",
            "max_35": "value_default_max_35",
        }
        if not isinstance(dict_scalars_to_val_ranges, dict)
        else dict_scalars_to_val_ranges
    )

    for k, v in dict_scalars_to_val_ranges.items():

        vec_scalar = np.array(df[k])
        vec_vals = np.array(df[field_final])
        df[v] = vec_scalar*vec_vals
        
    return df



def fp_template(x: str):
    return f"/Users/jsyme/Documents/Projects/git_jbus/sisepuede/ref/ingestion/demo/model_input_variables_{x}_demo.xlsx"



def get_sector(x: str):
    return sa.model_attributes.get_subsector_attribute(x, "sector")



########################
#    READ TEMPLATES    #
########################

dict_templates = {}
for subsec in ["af", "ce", "en", "ip", "se"]:
    dict_templates.update(
        {
            subsec: pd.read_excel(fp_template(subsec))
        }
    )


# build df
field_final = "35"
for k, v in dict_templates.items():
    
    fields_ext = [x for x in df_base.columns if x in v.columns] + [field_final]
    dict_templates.update({k: v[fields_ext]})

df_all_template_ranges = add_value_ranges(
    pd.concat(list(dict_templates.values()), axis = 0)
)

df_all_template_ranges["sector"] = list(df_all_template_ranges["subsector"].apply(get_sector))
df_all_template_ranges["include"] = 0
df_all_template_ranges.rename(
    columns = {field_final: f"value_default_nominal_{field_final}"},
    inplace = True,
)



fields_ord = [
    "sector",
    "subsector",
    "variable",
    "include",
    "variable_trajectory_group",
    "variable_trajectory_group_trajectory_type",
    "normalize_group",
    "trajgroup_no_vary_q",
    "uniform_scaling_q",
    f"min_{field_final}",
    f"max_{field_final}",
    f"value_default_nominal_{field_final}",
    f"value_default_min_{field_final}",
    f"value_default_max_{field_final}"
]

df_all_template_ranges = (
    df_all_template_ranges[fields_ord]
    .sort_values(by = fields_ord[0:3])
    .reset_index(drop = True)
)


sf.dict_to_excel(
    fp_uncertainty_definitions,
    {"definitions_default": df_all_template_ranges},
)

# Build a baseline design (2023071)


In [14]:
# variables to include

import sisepuede_models as sm
import warnings
warnings.filterwarnings("ignore")


models = sm.SISEPUEDEModels(
    sa.model_attributes,
    allow_electricity_run = False,
    fp_julia = sa.dir_jl,
    fp_nemomod_reference_files = sa.dir_ref_nemo,
)



In [48]:


modvars_include = [
    # AFOLU
    models.model_afolu.modvar_agrc_yf,
    models.model_afolu.modvar_lvst_carrying_capacity_scalar,
    models.model_afolu.modvar_lvst_equivalent_exports,
    models.model_afolu.modvar_lndu_ef_co2_conv,
    models.model_afolu.modvar_frst_sq_co2,
    
    # CircularEconomy
    
    # Energy
    models.model_energy.modvar_trde_demand_scalar,
    {
        models.model_energy.modvar_scoe_demscalar_elec_energy_demand: [
            "commercial_municipal",
            "residential"
        ],
    },
    
    {
        models.model_energy.modvar_scoe_demscalar_heat_energy_demand: [
            "commercial_municipal",
            "residential"
        ],
    },
    {
        models.model_energy.modvar_enfu_price_gravimetric: [
            "fuel_coke",
            "fuel_coal"
        ],
    },
    {
        models.model_energy.modvar_enfu_price_thermal: [
            "fuel_hydrogen",
            "fuel_hydrocarbon_gas_liquids",
            "fuel_natural_gas"
        ],
    },
    
    {
        models.model_energy.modvar_enfu_price_volumetric: [
             "fuel_crude",
             "fuel_diesel",
             "fuel_furnace_gas",
             "fuel_gasoline",
             "fuel_kerosene",
             "fuel_oil"
        ],
    
    },
    
    # IPPU
    models.model_ippu.modvar_ippu_scalar_production,
    
    # Socioeconomic
    models.model_socioeconomic.modvar_gnrl_climate_change_hydropower_availability
]


##  BUILD BASELINE VARIABLES TO INCLUDE

vars_include = []

for modvar_spec in modvars_include:
    
    if isinstance(modvar_spec, dict):
        for modvar, cats in modvar_spec.items():
            vars_include += models.model_attributes.build_varlist(
                None,
                modvar,
                restrict_to_category_values = cats,
            )
            
    elif isinstance(modvar_spec, str):
        vars_include += models.model_attributes.build_varlist(
            None,
            modvar_spec,
        )

vars_include.sort()


##  CONVERT TO DATA FRAME, MERGE, AND FILL

df_vars_include = pd.DataFrame({"variable": vars_include})
df_vars_include["include"] = 1


##  add in to data frame
df_all_template_ranges = (
    pd.merge(
        df_all_template_ranges.drop(["include"], axis = 1),
        df_vars_include,
        how = "left"
    )
    .fillna(0)
)
df_all_template_ranges["include"] = df_all_template_ranges["include"].astype("int64")



In [55]:
sf.dict_to_excel(
    fp_uncertainty_definitions,
    {"definitions_default": df_all_template_ranges},
)

In [58]:
df_all_template_ranges.to_csv(
    fp_uncertainty_definitions.replace(".xlsx", ".csv"),
    index = None,
    encoding = "UTF-8"
)

# Read in from CSV, then use to overwrite values in thhe demo templates

In [66]:
def fp_template(x: str):
    return f"/Users/jsyme/Documents/Projects/git_jbus/sisepuede/ref/ingestion/demo/model_input_variables_{x}_demo.xlsx"

fp_uncertainty_definitions = "/Users/jsyme/Documents/Projects/git_jbus/sisepuede/ref/ingestion/uncertainty_definitions/uncertainty_definitions_global.xlsx"

df_all_template_ranges = pd.read_csv(
    fp_uncertainty_definitions.replace(".xlsx", ".csv"),
)



dict_templates = {}
for subsec in ["af", "ce", "en", "ip", "se"]:
    dict_templates.update(
        {
            subsec: pd.read_excel(fp_template(subsec))
        }
    )
    



def set_ranges(
    df_template: pd.DataFrame,
    df_all_template_ranges: pd.DataFrame,
) -> pd.DataFrame:
    """
    Set ranges in df_template using information from df_all_template_ranges
    """
    
    field_include = "include"
    field_max = "max_35"
    field_min = "min_35"
    field_var = "variable"
    
    # get randge values to modify
    df_all_template_ranges_to_include = df_all_template_ranges[
        df_all_template_ranges["include"].isin([1])
    ]
    vars_keep = list(df_all_template_ranges_to_include[field_var].unique())
    
    # get replacement values 
    dict_var_to_max = sf.build_dict(df_all_template_ranges_to_include[[field_var, field_max]])
    dict_var_to_min = sf.build_dict(df_all_template_ranges_to_include[[field_var, field_min]])
    
    
    ##  SPLIT TEMPLATE INTO VARS TO INCLUDE (WITH RANGES) AND EXCLUDE (SET TO 1)
    
    # add ranges for variables that are to range
    df_template_incl = df_template[
        df_template[field_var].isin(vars_keep)
    ]
    df_template_incl[field_max] = df_template_incl[field_var].replace(dict_var_to_max)
    df_template_incl[field_min] = df_template_incl[field_var].replace(dict_var_to_min)
    
    # set to one otherwise (no uncertainty exploration)
    df_template_excl = df_template[
        ~df_template[field_var].isin(vars_keep)
    ]
    df_template_excl[field_max] = 1.0
    df_template_excl[field_min] = 1.0
    
    
    df_out = (
        pd.concat(
            [
                df_template_incl,
                df_template_excl
            ]
        )
        .sort_values(by = ["subsector", "variable"])
        .reset_index(drop = True)
    )
    
    return df_out





In [80]:
sheet_name = "strategy_id-0"
for k, v in dict_templates.items():
    df = v.copy()
    
    df_out = set_ranges(df, df_all_template_ranges)
    
    if df_out.shape == v.shape:
        sf.dict_to_excel(
            fp_template(k),
            {
                sheet_name: df_out
            }
        )
    else:
        print(f"Problem with subsector '{k}'")

In [79]:
fp_template("se")

'/Users/jsyme/Documents/Projects/git_jbus/sisepuede/ref/ingestion/demo/model_input_variables_se_demo.xlsx'

In [76]:
set_ranges(dict_templates.get("ip"), df_all_template_ranges)

,subsector,variable,variable_trajectory_group,variable_trajectory_group_trajectory_type,normalize_group,trajgroup_no_vary_q,uniform_scaling_q,min_35,max_35,0,...,26,27,28,29,30,31,32,33,34,35
0,IPPU,dem_ippu_average_materials_tonne_per_household...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1,IPPU,dem_ippu_average_materials_tonne_per_household...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.4,...,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4
2,IPPU,demscalar_ippu_cement,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,IPPU,demscalar_ippu_chemicals,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,IPPU,demscalar_ippu_construction_and_demolition,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,IPPU,vol_ippu_recycled_rubber_and_leather_m3_ww_per...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,16.0,...,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0
189,IPPU,vol_ippu_recycled_textiles_m3_ww_per_tonne_pro...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,16.0,...,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0
190,IPPU,vol_ippu_rubber_and_leather_m3_ww_per_tonne_pr...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,25.0,...,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0
191,IPPU,vol_ippu_textiles_m3_ww_per_tonne_production,NaN,NaN,NaN,NaN,NaN,1.0,1.0,25.0,...,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0


In [81]:
vars_include

['climate_change_factor_gnrl_hydropower_availability',
 'cost_enfu_fuel_coal_usd_per_tonne',
 'cost_enfu_fuel_coke_usd_per_tonne',
 'cost_enfu_fuel_crude_usd_per_m3',
 'cost_enfu_fuel_diesel_usd_per_m3',
 'cost_enfu_fuel_furnace_gas_usd_per_m3',
 'cost_enfu_fuel_gasoline_usd_per_m3',
 'cost_enfu_fuel_hydrocarbon_gas_liquids_usd_per_mmbtu',
 'cost_enfu_fuel_hydrogen_usd_per_mmbtu',
 'cost_enfu_fuel_kerosene_usd_per_m3',
 'cost_enfu_fuel_natural_gas_usd_per_mmbtu',
 'cost_enfu_fuel_oil_usd_per_m3',
 'demscalar_ippu_cement',
 'demscalar_ippu_chemicals',
 'demscalar_ippu_construction_and_demolition',
 'demscalar_ippu_electronics',
 'demscalar_ippu_glass',
 'demscalar_ippu_lime_and_carbonite',
 'demscalar_ippu_metals',
 'demscalar_ippu_mining',
 'demscalar_ippu_other_industries',
 'demscalar_ippu_other_product_manufacturing',
 'demscalar_ippu_paper',
 'demscalar_ippu_plastic',
 'demscalar_ippu_product_use_lubricants',
 'demscalar_ippu_product_use_ods_other',
 'demscalar_ippu_product_use_ods